# Домашнее задание по первой лекции.


## 1. Нидлман Вунш
Реализуйте алгоритм Нидлмана Вунша для выравнивания последовательностеей. На вход принимается две строки, матрица замен и стоимость гэпа. В результате верните оптимальное выравнивание и его вес. При проверке помните, что оптимальных выравниваний может быть несколько, но вес у них должен совпадать.

In [4]:
#from Bio.SubsMat import MatrixInfo as matlist
from Bio.Align import substitution_matrices
#matrix = matlist.blosum62 # Устаревший вариант
matrix = substitution_matrices.load('BLOSUM62')
print(matrix)
alfa = {"A", "T", "G", "C"}

nucleotides = ["A", "T", "G", "C"]
print("      A", "     T", "     G", "     C")
for i in nucleotides:
    print(i,end = ' ')
    for j in nucleotides:
        print(f"{matrix[i, j]:>6}", end = ' ')
    print()
new_matrix = {}


#  Matrix made by matblas from blosum62.iij
#  * column uses minimum score
#  BLOSUM Clustered Scoring Matrix in 1/2 Bit Units
#  Blocks Database = /data/blocks_5.0/blocks.dat
#  Cluster Percentage: >= 62
#  Entropy =   0.6979, Expected =  -0.5209
     A    R    N    D    C    Q    E    G    H    I    L    K    M    F    P    S    T    W    Y    V    B    Z    X    *
A  4.0 -1.0 -2.0 -2.0  0.0 -1.0 -1.0  0.0 -2.0 -1.0 -1.0 -1.0 -1.0 -2.0 -1.0  1.0  0.0 -3.0 -2.0  0.0 -2.0 -1.0  0.0 -4.0
R -1.0  5.0  0.0 -2.0 -3.0  1.0  0.0 -2.0  0.0 -3.0 -2.0  2.0 -1.0 -3.0 -2.0 -1.0 -1.0 -3.0 -2.0 -3.0 -1.0  0.0 -1.0 -4.0
N -2.0  0.0  6.0  1.0 -3.0  0.0  0.0  0.0  1.0 -3.0 -3.0  0.0 -2.0 -3.0 -2.0  1.0  0.0 -4.0 -2.0 -3.0  3.0  0.0 -1.0 -4.0
D -2.0 -2.0  1.0  6.0 -3.0  0.0  2.0 -1.0 -1.0 -3.0 -4.0 -1.0 -3.0 -3.0 -1.0  0.0 -1.0 -4.0 -3.0 -3.0  4.0  1.0 -1.0 -4.0
C  0.0 -3.0 -3.0 -3.0  9.0 -3.0 -4.0 -3.0 -3.0 -1.0 -1.0 -3.0 -1.0 -2.0 -3.0 -1.0 -1.0 -2.0 -2.0 -1.0 -3.0 -3.0 -2.0 -4.0
Q -1.0  1.0  0.0  0.

In [128]:
import numpy as np

def needleman_wunsch(seq1, seq2, subst_matrix, gap):
    n = len(seq1) + 1
    m = len(seq2) + 1
    
    dp = np.zeros((n, m))
    dp[1:, 0]= np.arange(1, n)*gap    
    dp[0, 1:]= np.arange(1, m)*gap
    
    for i in range(1, n):
        for j in range(1, m):
            match = dp[i-1][j-1] + subst_matrix[seq1[i-1]][seq2[j-1]]
            dlt = dp[i-1][j] + gap
            ins = dp[i][j-1] + gap
            dp[i][j] = max(match, dlt, ins)
    
    n -=1
    m -=1
        
    def get_alignments(i, j):
        if i == 0 and j == 0:
            return {("", "")}
        
        alignments = set()
        curr = dp[i, j]
                
        if i > 0 and j > 0 and curr == dp[i - 1, j - 1] + subst_matrix[seq1[i - 1]][seq2[j - 1]]:
            for alignment in get_alignments(i - 1, j - 1):
                alignments.add((alignment[0] + seq1[i - 1], alignment[1] + seq2[j - 1]))
                
        if i > 0 and curr == dp[i - 1, j] + gap:
            for alignment in get_alignments(i - 1, j):
                alignments.add((alignment[0] + seq1[i - 1], alignment[1] + "-"))
                
        if j > 0 and curr == dp[i, j - 1] + gap:
            for alignment in get_alignments(i, j - 1):
                alignments.add((alignment[0] + "-", alignment[1] + seq2[j - 1]))
        
        return alignments    

    #print(dp)
    
    return get_alignments(n, m), dp[n][m]    


### Проверка

In [267]:
seq1 = "GCATGCG"#"AGTACGCA"
seq2 = "CATTACA"#"TATGC"
seq1 = "GCA"#"AGTACGCA"
seq2 = "CATGA"#"TATGC"
seq1 = "GCGGGCG"#"AGTACGCA"
seq2 = "CCCTC"#"TATGC"
subst_matrix =matrix 
gap_penalty = -2

print('Данные для проверки:')
print('  Последовательность 1:', seq1)
print('  Последовательность 2:', seq2)
print('  Матрица замен:', 'BLOSUM62')
print('  Стоимость гэпа:', gap_penalty)
print('\nРезультат:')
i = 1
alignments, score = needleman_wunsch(seq1, seq2, subst_matrix, gap_penalty)
print("Score =", score)
for a, b in alignments:
    print(f"Optimal alignment {i}:")
    print(f'  {a}\n  {b}')
    i+=1

Данные для проверки:
  Последовательность 1: GCGGGCG
  Последовательность 2: CCCTC
  Матрица замен: BLOSUM62
  Стоимость гэпа: -2

Результат:
[[  0.  -2.  -4.  -6.  -8. -10.]
 [ -2.  -3.  -5.  -7.  -8. -10.]
 [ -4.   7.   6.   4.   2.   1.]
 [ -6.   5.   4.   3.   2.   0.]
 [ -8.   3.   2.   1.   1.  -1.]
 [-10.   1.   0.  -1.  -1.  -2.]
 [-12.  -1.  10.   9.   7.   8.]
 [-14.  -3.   8.   7.   7.   6.]]
Score = 6.0
Optimal alignment 1:
  GCGGGCG
  CC-CTC-
Optimal alignment 2:
  GCGGGCG
  CCCT-C-
Optimal alignment 3:
  GCGGGCG
  -CCCTC-
Optimal alignment 4:
  GCGGGCG
  CCC-TC-


## 2. Афинные гэпы
Реализуйте выравнивание с афинными гэпами, алгоритм на вход принимает две строки, матрицу замен, штраф за начало гэпа α, и за его продолжение β. В результате возвращает выравнивание и его вес. Сложность алгоритма квадратичная по памяти и по времени

In [2]:
import numpy as np

def affine_alignment(seq1, seq2, subst_matrix, gap_a, gap_b):
    n = len(seq1) + 1
    m = len(seq2) + 1
    if n == m == 1: return set(), 0
    
    dp =[[[-np.inf for _ in range(3) ] for j in range(m)] for i in range(n)]
    #gap_beg = np.full((n, m), -np.inf)#np.zeros((n, m))
    #gap_next = np.full((n, m ), -np.inf)#np.zeros((n, m))
    #gap_beg[0, 0], gap_next[0,0] = 0,0     
    dp[0][0] = [0,0,0]
    
    cost = gap_a
    for i in range(1, n):        
        #dp[i][0] = cost
        if i == 1: dp[i][0][1] = cost
        else: dp[i][0][2] = cost
        cost +=gap_b        

    cost = gap_a
    for j in range(1, m):            
        if j == 1: dp[0][j][1] = cost
        else: dp[0][j][2] = cost
        cost +=gap_b
    
    
    for i in range(1, n):
        for j in range(1, m ):                        
            match =  subst_matrix[seq1[i - 1]][seq2[j - 1]] + max(dp[i - 1][j - 1])
            dp[i][j][0] = match 
            dp[i][j][1]  = max(dp[i][j - 1][0], dp[i-1][j ][0])+gap_a
            dp[i][j][2]  = max(dp[i][j - 1][1], dp[i-1][j ][1])+gap_b           

    n -=1
    m -=1
    mcurr = max(dp[n][m])
    
    def get_alignments(i, j, curr):        
        if i == 0 and j == 0:
            return {("", "")}
        
        alignments = set()
                
        if i > 0 and j > 0 and curr == max(dp[i - 1][ j - 1]) + subst_matrix[seq1[i - 1]][seq2[j - 1]]:
            for alignment in get_alignments(i - 1, j - 1, max(dp[i - 1][ j - 1])):
                alignments.add((alignment[0] + seq1[i - 1], alignment[1] + seq2[j - 1]))        

        if i > 0 and curr == dp[i - 1][ j][0] + gap_a == dp[i][j][1]:
            for alignment in get_alignments(i - 1, j, dp[i - 1][ j][0]):
                alignments.add((alignment[0] + seq1[i - 1], alignment[1] + "-"))
                
        if j > 0 and curr == dp[i][j - 1][0] + gap_a == dp[i][j][1]:
            for alignment in get_alignments(i, j - 1, dp[i][j - 1][0]):
                alignments.add((alignment[0] + "-", alignment[1] + seq2[j - 1]))

        if curr == dp[i][j][2]:
            
            if i > 0 and (curr == dp[i - 1][ j][1] + gap_b ):
                for alignment in get_alignments(i - 1, j, dp[i - 1][ j][1]):
                    alignments.add((alignment[0] + seq1[i - 1], alignment[1] + "-"))
            if i > 0 and ( curr == dp[i - 1][ j][2] + gap_b):
                for alignment in get_alignments(i - 1, j, dp[i - 1][ j][2]):
                    alignments.add((alignment[0] + seq1[i - 1], alignment[1] + "-"))
                    
            if j > 0 and (curr == dp[i][ j - 1][1] + gap_b):
                for alignment in get_alignments(i, j - 1, dp[i][ j - 1][1]):
                    alignments.add((alignment[0] + "-", alignment[1] + seq2[j - 1]))
            if j > 0 and (curr == dp[i][ j - 1][2] + gap_b):
                for alignment in get_alignments(i, j - 1, dp[i][ j - 1][2]):
                    alignments.add((alignment[0] + "-", alignment[1] + seq2[j - 1]))
        
        return alignments 

    return get_alignments(n, m, mcurr), mcurr


### Проверка

In [5]:
seq1 = "GCA"
seq2 = "TGCGC"
subst_matrix =matrix 
gap_a = -3
gap_b = -1

print('Данные для проверки:')
print('  Последовательность 1:', seq1)
print('  Последовательность 2:', seq2)
print('  Матрица замен:', 'BLOSUM62')
print('  Штраф за начало гэпа:', gap_a)
print('  Штраф за продолжение гэпа:', gap_b)
print('\nРезультат:')
i = 1
alignments, score = affine_alignment(seq1, seq2, subst_matrix, gap_a, gap_b)
print("Score =", score)
for a, b in alignments:
    print(f"Optimal alignment {i}:")
    print(f'  {a}\n  {b}')
    i+=1


Данные для проверки:
  Последовательность 1: GCA
  Последовательность 2: TGCGC
  Матрица замен: BLOSUM62
  Штраф за начало гэпа: -3
  Штраф за продолжение гэпа: -1

Результат:
Score = 9.0
Optimal alignment 1:
  -GC-A
  TGCGC
Optimal alignment 2:
  -GCA-
  TGCGC


In [392]:
#Использовала для тестирования и сравнения результатов
from Bio import pairwise2

seq1 = "GCA"
seq2 = "TGCGC"

alignments = pairwise2.align.globalds(seq1, seq2, matrix, -3, -1)

for alignment in alignments:
    print(pairwise2.format_alignment(*alignment))


-GCA-
 ||. 
TGCGC
  Score=9

-GC-A
 || .
TGCGC
  Score=9



## 3. Теоретическая часть
Количество выравниваний.

Выведите рекуррентную формулу количества всех возможных выравниваний последовательностей длины n и m пользуясь разбиением всех выравниваний на непересекающиеся блоки. 

Получите точную формулу, основываясь на начальные условия и рекуррентную формулу.

Воспользуйтесь приближением Стирлинга чтобы получить приближенную формулу количества выравниваний.

**Выведите рекуррентную формулу количества всех возможных выравниваний последовательностей длины n и m пользуясь разбиением всех выравниваний на непересекающиеся блоки.**

Начальные условия:
- W(0, 0) = 1
- W(n, 0) = 1
- W(0, m) = 1

Рекуррентная формула:
$$ W(n, m) = W(n-1, m-1) + W(n-1, m) + W(n, m-1) $$
Где:
- W(n-1, m-1) = сопоставление символов двух строк
- W(n-1, m) = гэп во второй последовательности 
- W(n, m-1) = гэп в первой последовательности

**Получите точную формулу, основываясь на начальные условия и рекуррентную формулу**

Формулу нашла в статье https://doi.org/10.1080/10425170310001617894

$$ f(n, m) = \sum_{k=0}^{\min(n, m)} 2^k\binom{m}{k} \binom{n}{k} $$ 

**Воспользуйтесь приближением Стирлинга чтобы получить приближенную формулу количества выравниваний**
$$ f(n,m) =  \sum_{k=0}^{\min(n, m)}2^k \binom{m}{k} \binom{n}{k}  =  \sum_{k=0}^{\min(n, m)}\frac{2^k m!  n!}{(k!)^2 (n-k)! (m-k)!} $$
$$n! \sim \sqrt{2 \pi n} \left( \frac{n}{e} \right)^n $$
$$S(k) = \frac{2^k\cdot 
\sqrt{2\pi m} \left(\frac{m}{e}\right)^m \cdot 
\sqrt{2\pi n} \left(\frac{n}{e}\right)^n 
}{
2\pi k \left(\frac{k}{e}\right)^{2k} \cdot 
\sqrt{2\pi(n-k)} \left(\frac{n-k}{e}\right)^{n-k} \cdot 
\sqrt{2\pi(m-k)} \left(\frac{m-k}{e}\right)^{m-k}
} = \frac{2^k\cdot m^{\frac{1}{2}+m} \cdot n^{\frac{1}{2}+n} \cdot e^{2k+m-k+n-k}
}{
2\pi k \cdot k^{2k} \cdot  (n-k)^{\frac{1}{2}+n-k} \cdot  ({m-k})^{\frac{1}{2}+m-k}\cdot e^{m+n}
}$$
$$=\frac{2^k\cdot m^{\frac{1}{2}+m} \cdot n^{\frac{1}{2}+n} 
}{
2\pi \cdot k^{2k+1} \cdot  (n-k)^{\frac{1}{2}+n-k} \cdot  ({m-k})^{\frac{1}{2}+m-k}
}
$$
$$ f(n,m) \approx  \sum_{k=0}^{\min(n, m)}\frac{2^k\cdot m^{\frac{1}{2}+m} \cdot n^{\frac{1}{2}+n} 
}{
2\pi \cdot k^{2k+1} \cdot  (n-k)^{\frac{1}{2}+n-k} \cdot  ({m-k})^{\frac{1}{2}+m-k}\cdot
} $$